In [1]:
from PandaStation import *
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG



import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    LoadModelDirectives, ProcessModelDirectives, ConnectMeshcatVisualizer, Diagram
    )
import pydrake.perception as mut
import open3d as o3d
from ompl import base as ob
from ompl import geometric as og
import time
from enum import Enum

def ycb_resource(name):
    ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
        "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
        "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
        "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
        "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
        "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
        "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}
    return FindResourceOrThrow(ycb[name])

In [2]:
def draw_grasp_candidate(X_WG, prefix='gripper'):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    gripper = AddPandaHand(plant, welded = True)
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("panda_hand"), X_WG)
    plant.Finalize()
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix=prefix, delete_prefix_on_load=False)#, frames_to_draw=frames_to_draw)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    meshcat.load()
    diagram.Publish(context)
    

def draw_grasp_and_arm_candidate(X_WG, prefix = 'arm'):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    panda = AddPanda(plant)
    gripper = AddPandaHand(plant, panda_model_instance = panda, welded = True)
    plant.Finalize()
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix=prefix, delete_prefix_on_load=False)#, frames_to_draw=frames_to_draw)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    q = X_WG_to_q(plant, plant_context, X_WG)
    plant.SetPositions(plant_context, panda, q)
    meshcat.load()
    diagram.Publish(context)
    
def X_WG_to_q(plant, plant_context, X_G_lower, X_G, X_G_upper, no_cols = False, warm_start = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])):
    ik = InverseKinematics(plant, plant_context)
    p_tol = np.ones(3)*0.01
    theta_tol = 0.01
    q_nominal = np.array([ 0., 0.55, 0., -1.45, 0., 1.58, 0.])
    
    trans_lower = []
    trans_upper = []
    
    for i in range(3):
        trans_lower.append(min(X_G_lower.translation()[i], X_G_upper.translation()[i]))
        trans_upper.append(max(X_G_lower.translation()[i], X_G_upper.translation()[i]))
    
    ik.AddPositionConstraint(
            plant.GetFrameByName("panda_hand"),
            np.zeros(3),
            plant.world_frame(),
            trans_lower,
            trans_upper)
    ik.AddOrientationConstraint(
            plant.GetFrameByName("panda_hand"),
            RotationMatrix(),
            plant.world_frame(),
            X_G.rotation(),
            theta_tol)
    if no_cols:
        ik.AddMinimumDistanceConstraint(0, 0.1)
        
    q = ik.q()
    prog = ik.prog()
    prog.AddQuadraticErrorCost(np.identity(len(q)), q_nominal, q)
    prog.SetInitialGuess(q, warm_start)

    result = Solve(prog)
    #assert result.is_success()
    print("Success?", result.is_success())
    if not result.is_success():
        names = result.GetInfeasibleConstraintNames(prog)
        print("Infeasible Constraints:")
        for n in names:
            print(n)
    return result.GetSolution(q)

def process_table_point_cloud(env, env_context):
    
    plant = env.get_multibody_plant()
    plant_context = env.GetSubsystemContext(plant, env_context)

    margin = 0.001
    # defines space above table
    crop_min = np.array([0.25, -0.5, 0.05+margin])
    crop_max = np.array([1.25, 0.5, 0.5])

    merged_pcd = o3d.geometry.PointCloud()
    
    cameras = ['camera_left', 'camera_middle', 'camera_right']

    for name in cameras:
        pcd = env.GetOutputPort(f"{name}_point_cloud").Eval(env_context)
        pcd = create_open3d_point_cloud(pcd)

        pcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound = crop_min, max_bound = crop_max))
        pcd.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius = 0.1, max_nn = 30))
        camera = plant.GetModelInstanceByName(name)
        body = plant.GetBodyByName("base", camera)
        X_C = plant.EvalBodyPoseInWorld(plant_context, body)
        pcd.orient_normals_towards_camera_location(X_C.translation())

        merged_pcd +=pcd

    return merged_pcd.voxel_down_sample(voxel_size = 0.005)

In [7]:
rng = np.random.default_rng()

builder = DiagramBuilder()
station = builder.AddSystem(PandaStation())
station.SetupTableStation(welded_hand = True)
station.AddModelFromFile(ycb_resource("soup"), RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0.6, 0, 0.1]))
station.Finalize()
scene_graph = station.get_scene_graph()
v = ConnectMeshcatVisualizer(builder,
                          scene_graph,
                          output_port=station.GetOutputPort("query_object"),
                          delete_prefix_on_load=True,                                      
                          zmq_url=zmq_url)#, role = Role.kProximity)
v.load()
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
diagram.Publish(diagram_context)
station_context = station.GetMyContextFromRoot(diagram_context)

hand_env = station.get_hand_env() # environemnt with only the hand
hand_env_context = hand_env.CreateDefaultContext()

cloud = process_table_point_cloud(station, station_context)
draw_open3d_point_cloud(v.vis["cloud"], cloud, size=0.003, normals_scale = 0.01)
num_samples = 100

plant = station.get_multibody_plant()
plant_context = station.GetSubsystemContext(plant, station_context)

costs = []
X_Gs = []

for i in range(num_samples):
    cost, X_G_lower, X_G, X_G_upper = generate_grasp_candidate_antipodal(hand_env, hand_env_context, cloud, rng)
    if np.isfinite(cost):
        costs.append(cost)
        X_Gs.append((X_G_lower, X_G, X_G_upper))

indices = np.asarray(costs).argsort()[:1]
index = indices[0]
X_G_lower = X_Gs[index][0]
X_G = X_Gs[index][1]
X_G_upper = X_Gs[index][2]

'''
for i, index in enumerate(indices):
    draw_grasp_candidate(X_Gs[index][0], prefix=f"{i}th best hand lower")
    draw_grasp_candidate(X_Gs[index][1], prefix=f"{i}th best hand")
    draw_grasp_candidate(X_Gs[index][2], prefix=f"{i}th best hand upper")
    #draw_grasp_and_arm_candidate(X_Gs[index], prefix=f"{i}th best")
'''

ik_station = builder.AddSystem(PandaStation())
ik_station.SetupTableStation(welded_hand = True)
ik_station.Finalize()

plant = ik_station.get_multibody_plant()
ik_station_context = ik_station.CreateDefaultContext()
plant_context = ik_station.GetSubsystemContext(plant, ik_station_context)

q = X_WG_to_q(plant, plant_context, X_G_lower, X_G, X_G_upper, no_cols = True)
print(q)
station.SetPandaPosition(station_context, q)
diagram.Publish(diagram_context)


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
Success? True
[-0.02823459  0.39291921  0.01187307 -2.0271851   0.07030897  2.40850494
 -0.05937915]
